# Inputting only s11 and freq and outputting all geometry parameters

### Regular TensorFlow 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from matplotlib import rc

/tmp/ipykernel_554781/3602695761.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-26 01:41:26.492240: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 01:41:26.550876: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 01:4

In [2]:
# Import data and preprocess
df = pd.read_csv("../test_data/new leaky wave/S11_Data_combined.csv")
# df = pd.read_csv("../test_data/Grounded CPW Leaky Wave antenna/S11 Data.csv")
df = df.drop(df[df['dB(S(1,1)) []'] > 0].index) # Remove all rows with positive s11

In [3]:


# # Split into x and y
# input_x = df.drop(columns=['dB(S(1,1)) []'], axis=1)
# input_y = df[['dB(S(1,1)) []']]

# Split data into training and testing
train_df = df.sample(frac=0.8, random_state=0)
test_df = df.drop(train_df.index)
training_cols = ['dB(S(1,1)) []', 'Freq [GHz]']
X_train = train_df[training_cols]
X_test = test_df[training_cols]
y_train = train_df.drop(columns=training_cols)
y_test = test_df.drop(columns=training_cols)

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # Initialize scaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
X_train

,"dB(S(1,1)) []",Freq [GHz]
5904,-10.129858,15.14
2298,-4.979113,17.84
15477,-10.679959,13.16
13716,-13.218871,18.29
18849,-5.414222,16.67
...,...,...
5277,-13.306406,13.25
23293,-13.808419,16.67
6103,-10.716262,14.87
20521,-11.720165,12.62


In [6]:
y_train

,cpw_in [mm],feed_l [mm],patch_l [mm],cpw_g [mm],Feed_W [mm],ground_w [mm],patch_ground_w [mm],patch_w [mm]
5904,2.25,3.50,3.75,0.18,1.0,1.00,1.0,4.75
2298,2.00,4.25,3.25,0.18,1.0,1.00,1.0,4.75
15477,2.00,3.75,3.50,1.02,2.0,1.00,1.0,4.75
13716,2.00,3.75,3.50,0.42,1.0,1.00,1.0,4.75
18849,2.00,3.75,3.50,2.40,1.5,1.00,1.0,4.75
...,...,...,...,...,...,...,...,...
5277,2.00,3.25,3.75,0.18,1.0,1.00,1.0,4.75
23293,2.00,3.75,3.50,0.18,1.0,1.25,1.0,5.00
6103,1.50,3.75,3.75,0.18,1.0,1.00,1.0,4.75
20521,2.00,3.75,3.50,0.18,1.0,1.50,0.5,5.00


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

inputs = Input(shape=(2,))
hidden1 = Dense(160, activation='relu')(inputs)
hidden2 = Dense(160, activation='relu')(hidden1)
hidden3 = Dense(160, activation='relu')(hidden2)
hidden4 = Dense(160, activation='relu')(hidden3)

outputs = Dense(8, activation='linear')(hidden4)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

2024-03-26 01:41:30.767676: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1030 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 160)               480       
                                                                 
 dense_1 (Dense)             (None, 160)               25760     
                                                                 
 dense_2 (Dense)             (None, 160)               25760     
                                                                 
 dense_3 (Dense)             (None, 160)               25760     
                                                                 
 dense_4 (Dense)             (None, 8)                 1288      
                                                                 
Total params: 79048 (308.78 KB)
Trainable params: 79048 (308.

In [8]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


history = model.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/150


2024-03-26 01:41:33.444362: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fd8309bbce0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-26 01:41:33.444413: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-26 01:41:33.452124: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-26 01:41:33.504232: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1711431693.713781  554892 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


534/534 [==============================] - 7s 5ms/step - loss: 0.3002 - accuracy: 0.9930 - val_loss: 0.0765 - val_accuracy: 1.0000
Epoch 2/150
534/534 [==============================] - 3s 5ms/step - loss: 0.0760 - accuracy: 1.0000 - val_loss: 0.0791 - val_accuracy: 1.0000
Epoch 3/150
534/534 [==============================] - 3s 5ms/step - loss: 0.0755 - accuracy: 1.0000 - val_loss: 0.0759 - val_accuracy: 1.0000
Epoch 4/150
534/534 [==============================] - 3s 5ms/step - loss: 0.0751 - accuracy: 1.0000 - val_loss: 0.0733 - val_accuracy: 1.0000
Epoch 5/150
534/534 [==============================] - 3s 5ms/step - loss: 0.0749 - accuracy: 1.0000 - val_loss: 0.0770 - val_accuracy: 1.0000
Epoch 6/150
534/534 [==============================] - 3s 5ms/step - loss: 0.0745 - accuracy: 1.0000 - val_loss: 0.0775 - val_accuracy: 1.0000
Epoch 7/150
534/534 [==============================] - 3s 5ms/step - loss: 0.0739 - accuracy: 1.0000 - val_loss: 0.0714 - val_accuracy: 1.0000
Epoch 8/150

In [9]:
predictions = model.predict(X_test_scaled)

167/167 [==============================] - 1s 2ms/step


In [10]:
pd.DataFrame(predictions)

,0,1,2,3,4,5,6,7
0,2.040600,3.658811,3.536693,0.694509,1.037813,0.899840,1.101839,4.765162
1,2.013436,3.667971,3.535750,1.466804,1.077816,1.015744,1.039038,4.741011
2,2.091075,3.674082,3.594768,0.955140,1.170641,1.025676,0.948949,4.737618
3,2.070843,3.644548,3.565660,0.809455,1.160403,0.995074,0.977010,4.701676
4,1.988813,3.731130,3.630625,0.482569,1.151716,1.010889,1.028888,4.707995
...,...,...,...,...,...,...,...,...
5328,1.995808,3.778680,3.553374,0.428548,1.077090,1.018338,1.068352,4.719641
5329,1.937728,3.691396,3.633530,0.251812,1.053523,1.027642,1.017268,4.717072
5330,2.031571,3.797565,3.565316,0.501300,1.055941,1.023673,1.053050,4.732848
5331,1.969893,3.735552,3.555073,0.521882,1.065517,0.985629,1.026146,4.707335


In [11]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, predictions)

0.06807979621512608

### Keras Tuner

In [12]:
import keras_tuner
class AntennaModel(keras_tuner.HyperModel):
    def __init__(self, input_shape):
        self.input_shape= input_shape
        
    def build(self, hp):
        model= keras.Sequential()

        for i in range(hp.Int('num_layers', 1, 4)):
            model.add(keras.layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation='relu', input_shape=self.input_shape))
        model.add(keras.layers.Dense(8)) # Output layer

        # Tune the learning rate for the optimizer 
        hp_learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=0.1)

        model.compile(loss='mse',
                    optimizer= keras.optimizers.Adam(learning_rate=hp_learning_rate),
                    metrics= ['mse']
                     )

        return model

In [13]:
import keras_tuner
antenna_model = AntennaModel(input_shape=[len(X_train.keys())])
tuner = keras_tuner.RandomSearch(
    antenna_model,
    objective='val_mse',
    max_trials=3,
    executions_per_trial=1)

tuner.search_space_summary()
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train_scaled, y_train, epochs=100, validation_split=0.2, validation_data = (X_test_scaled, y_test), callbacks=[stop_early])

Trial 3 Complete [00h 00m 26s]
val_mse: 0.07946301996707916

Best val_mse So Far: 0.07434263080358505
Total elapsed time: 00h 01m 47s


In [14]:
best_model = tuner.get_best_models(num_models=1)[0]
tuner_preds = best_model.predict(X_test_scaled)
mean_squared_error(y_test, tuner_preds)


167/167 [==============================] - 1s 2ms/step


0.07434262287474501

### Sklearn

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

model = MultiOutputRegressor(
    RandomForestRegressor(max_depth=85, min_samples_leaf=1, min_samples_split=2, n_estimators=1000
))
scaler = StandardScaler() # Initialize scaler
pipeline = Pipeline(steps=[('normalize', scaler), ('model', model)]) # Create pipeline with scaler and model


In [16]:
pipeline.fit(X_train_scaled,y_train)

Pipeline(steps=[('normalize', StandardScaler()),
                ('model',
                 MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=85,
                                                                      n_estimators=1000)))])

In [17]:
sklearn_pred = pipeline.predict(X_test_scaled)

In [18]:
mean_squared_error(y_test, sklearn_pred)

0.08118084632368677